### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [26]:
#b)
baby_df["review"] = baby_df["review"].fillna("")
#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [27]:
#a)
baby_df["review"] = baby_df["review"].apply(remove_punctuation)
#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock.,'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [28]:
#c)
baby_df = baby_df[baby_df["rating"] != 3]
#short test:
sum(baby_df["rating"] == 3)

0

In [29]:
#d) 
baby_df["rating"] = baby_df["rating"].apply(lambda x: 1 if x >= 4 else -1)
#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [7]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [31]:
#a)
train_df, test_df = train_test_split(baby_df, test_size=0.2, random_state=42)

In [38]:
#b)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df["review"])
print(vectorizer.get_feature_names_out())

['00' '000' '0001' ... 'zzzzzz' 'zzzzzzz' 'zzzzzzzzzzz']


## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [39]:
#a)
model = LogisticRegression()
model.fit(X_train, train_df["rating"])


C:\Users\laptopBJ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [45]:
#b)
print(model.coef_)
coefficients = model.coef_[0]
words = vectorizer.get_feature_names_out()
sorted_names = np.argsort(coefficients)
print("10 most negative words:")
for i in range(10):
    print(words[sorted_names[i]])
print("10 most positive words:")
for i in range(10):
    print(words[sorted_names[-i]])
#hint: model.coef_, vectorizer.get_feature_names()

[[ 0.00130192  0.00403392  0.01242036 ...  0.00132663  0.00077475
  -0.00039538]]
10 most negative words:
worst
disappointing
useless
poorly
poor
terrible
concept
returning
returned
horrible
10 most positive words:
excellent
awesome
pleased
worry
amazing
glad
satisfied
complaint
highly
exactly


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [47]:
#a)
y_pred = model.predict(vectorizer.transform(test_df["review"]))
print(y_pred)

[ 1 -1 -1 ...  1  1  1]


In [49]:
#b)
y_pred_proba = model.predict_proba(vectorizer.transform(test_df["review"]))
print(y_pred_proba)
#hint: model.predict_proba()

[[3.98265877e-01 6.01734123e-01]
 [7.96821619e-01 2.03178381e-01]
 [7.02820286e-01 2.97179714e-01]
 ...
 [6.91543053e-06 9.99993085e-01]
 [2.18544634e-04 9.99781455e-01]
 [8.32979911e-02 9.16702009e-01]]


In [50]:
#c) 
best_reviews = np.argsort(y_pred_proba[:,5])
worst_reviews = np.argsort(y_pred_proba[:,5])
print("5 most positive reviews:")
for i in range(5):
    print(test_df["review"][best_reviews[i]])

print("5 most negative reviews:")
for i in range(5):
    print(test_df["review"][worst_reviews[i]])
#hint: use the results of b)

IndexError: index 5 is out of bounds for axis 1 with size 2

In [15]:
#d) 


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [ ]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [16]:
#a)


In [17]:
#b)


In [18]:
#c)

#hint: %time, %timeit